In [1]:
from collections import Counter

def affine_decrypt(a_inv, y, m):
    x = a_inv * y % m
    
    return x

In [2]:
def create_frequency(ciphertext, m=2):
    digrams_counts = Counter(map(''.join, zip(ciphertext, ciphertext[1:])))
    return digrams_counts

In [3]:
def next_max_idx(arr, curr_max_idx):
    new_max_idx = 0
    temp_min_val = 0
    
    for i in range(len(arr)):
        if arr[i] > temp_min_val and (curr_max_idx == 26 or arr[i] < arr[curr_max_idx]):
            new_max_idx = i
            temp_min_val = arr[new_max_idx]
    
    return new_max_idx

def decrypt_message(y, a_inv, m):
    
    x = ""
     
    for i in range(len(y)):
        #First 2 characters of the cipher text
        cipher_bigram_char1 = ord(y[i][0]) - ord('a')     # ord('a') = 97
        cipher_bigram_char2 = ord(y[i][1]) - ord('a')     # ord('a') = 97
        
        y_num = vector([cipher_bigram_char1, cipher_bigram_char2])
        
        x_num = affine_decrypt(a_inv, y_num, m)
        
        #append decrypted characters into the value x
        x     = x + (chr(int(x_num[0]) + ord('a')) + chr(int(x_num[1]) + ord('a')))
        #print(x)

    return x
        
def inv(a, det):
    #FIND THE MULTIPLICATIVE INVERSE
    
    adj = a.adjugate()
    for x in range(1,26):
        if(((det % 26) * (x%26)) % 26 == 1):
            #multiply adjacate with the mul_inv and mod 26
            #Giving us the inverse of the key
            inv = (adj * x) % 26
            return inv
    
    #print("asdfasdf:",mul_inv)

    return -1
    #return a.inverse()

def solve(x1_1, x1_2, x2_1, x2_2, y1, y2, y3, y4, m):
    
    #2 Systems of equations to find the key
    
    Zm=Integers(m) # Mod 26
    MS_Zm1 = MatrixSpace(Zm, 2, 2) # '2, 2' refer to number of rows and columns of the matrices represented by this space
    MS_Zm2 = MatrixSpace(Zm, 2, 2)
    MS_Key = MatrixSpace(QQ, 2, 2)
    
    
    A_zm1 = MS_Zm1([x1_1,x1_2,x2_1,x2_2])  # we solve for a and b in a*x1+b=y1; a*x2+b=y2

    Y_zm1 = vector(Zm, [y1, y3])
    Y_zm2 = vector(Zm, [y2, y4])
    
    X_zm1 = A_zm1 \ Y_zm1
    X_zm2 = A_zm1 \ Y_zm2
    
    key = MS_Key([X_zm1[0],X_zm1[1],X_zm2[0],X_zm2[1]])
    print('key:',X_zm1[0],X_zm1[1],X_zm2[0],X_zm2[1])
    return key

In [4]:
m = 26
y = "LMQETXYEAGTXCTUIEWNCTXLZEWUAISPZYVAPEWLMGQWYAXFTCJMSQCADAGTXLMDXNXSNPJQSYVAPRIQSMHNOCVAXFV".lower()
x = ""
digram_freq = [x.lower() for x in ["TH", "HE", "IN", "ER", "AN", "RE", "ED", "ON", "ES", "ST"]]

#Seaparated Characters of the cipher text
cipher_text_bigrams=[y[i:i+2] for i in range(0, len(y),2)]

#Frequency List
freq = create_frequency(y)
freq = sorted(freq.items(), key=lambda x: x[1], reverse=True)

prev_high_freq_ciphertext_lttr = -1

for i in range(len(digram_freq)):

    #First Digram Pair
    digram_pair = digram_freq[i]
    
    digram_high_freq_char1 = digram_pair[0]
    high_freq_plaintext_lttr1 = ord(digram_high_freq_char1) - ord('a')
    
    digram_high_freq_char2 = digram_pair[1]
    high_freq_plaintext_lttr2 = ord(digram_high_freq_char2) - ord('a')
    
    for j in range(len(digram_freq)): # for j in [i+1 .. m-1]:
        
        if j != i:
            
            #Second Digram Pair
            
            next_digram_pair = digram_freq[j]
    
            next_digram_high_freq_char1 = next_digram_pair[0]
            next_high_freq_plaintext_lttr1 = ord(next_digram_high_freq_char1) - ord('a')
    
            next_digram_high_freq_char2 = next_digram_pair[1]
            next_high_freq_plaintext_lttr2 = ord(next_digram_high_freq_char2) - ord('a')
            
            
            #Print highest freq pair
            print("plaintext higher freq two digrams are: {" + str(digram_freq[i]) + ", " + str(digram_freq[j]) + "}\n")

            high_freq_ciphertext_lttr = 0 # prev_high_freq_ciphertext_lttr + 1 # next_max_idx(freq, prev_high_freq_ciphertext_lttr) # return the idx of the next max in the list of freq. This is the Z_26 representation of the highest freq ciphertext letter
            
            while high_freq_ciphertext_lttr < 26 and freq[high_freq_ciphertext_lttr][1] > 0:  # if freq of the digram is > 0

                next_high_freq_ciphertext_lttr = 0 # high_freq_ciphertext_lttr + 1 # next_max_idx(freq, high_freq_ciphertext_lttr)    # second next max freq ciphertext letter

                while next_high_freq_ciphertext_lttr < 26 and freq[next_high_freq_ciphertext_lttr][1] > 0:

                    if next_high_freq_ciphertext_lttr != high_freq_ciphertext_lttr:
                        print("ciphertext higher freq two letters are: {" + freq[high_freq_ciphertext_lttr][0] + ", " + freq[next_high_freq_ciphertext_lttr][0] + "}\n")
                        
                        # unicode values of the cipher texts
                        y1 = ord(freq[high_freq_ciphertext_lttr][0][0]) - ord('a')
                        y2 = ord(freq[high_freq_ciphertext_lttr][0][1]) - ord('a')
                        y3 = ord(freq[next_high_freq_ciphertext_lttr][0][0]) - ord('a')
                        y4 = ord(freq[next_high_freq_ciphertext_lttr][0][1]) - ord('a')
                        
                        
                        solution = solve(high_freq_plaintext_lttr1, high_freq_plaintext_lttr2, next_high_freq_plaintext_lttr1, next_high_freq_plaintext_lttr2, y1,y2,y3,y4, m)

                        k00 = int(solution[0][0])
                        k01 = int(solution[0][1])
                        k10 = int(solution[1][0])
                        k11 = int(solution[1][1])
                        
                        #print(k00,k01,k10,k11)
                    
                        # Find if inverse or not (determinant not = 0)
                        det = k00*k11 - k01*k10
                        #print("det", det)                        
                        
                        det = det % 26
                        
                        if(solution.is_invertible() == True and inv(solution,det) != True):
                            if(k00 == 4 and k01 == 13 and k10 == 11 and k11 == 9):
                                a_inv = inv(solution, det)
                            #print("matrx",solution)
                                x = decrypt_message(cipher_text_bigrams, a_inv, m)                    
                                print("plaintext = ", x)
                            #print("a_inv = " + str(a_inv) + ", b = " + str(b) + ", plaintext = " + x)
                            #ch = input("Press Enter to continue or E to exit...")
                        else:
                            print("Detemerinant is 0: is not a valid solution\n")

                    next_high_freq_ciphertext_lttr = next_high_freq_ciphertext_lttr + 1 # next_max_idx(freq, next_high_freq_ciphertext_lttr)

                high_freq_ciphertext_lttr = high_freq_ciphertext_lttr + 1 # next_max_idx(freq, high_freq_ciphertext_lttr)

plaintext higher freq two digrams are: {th, he}

ciphertext higher freq two letters are: {tx, lm}

key: 25 24 8 15
ciphertext higher freq two letters are: {tx, ew}

key: 22 21 16 23
ciphertext higher freq two letters are: {tx, va}

key: 7 6 14 21
ciphertext higher freq two letters are: {tx, ag}

key: 24 23 24 5
ciphertext higher freq two letters are: {tx, gt}

key: 8 7 11 18
ciphertext higher freq two letters are: {tx, ct}

key: 10 9 11 18
ciphertext higher freq two letters are: {tx, xl}

key: 19 18 15 22
ciphertext higher freq two letters are: {tx, yv}

key: 12 11 23 4
ciphertext higher freq two letters are: {tx, ap}

key: 24 23 13 20
ciphertext higher freq two letters are: {tx, ax}

key: 24 23 9 16
ciphertext higher freq two letters are: {tx, xf}

key: 19 18 5 12
ciphertext higher freq two letters are: {tx, qs}

key: 16 15 18 25
ciphertext higher freq two letters are: {tx, mq}

key: 18 17 6 13
ciphertext higher freq two letters are: {tx, qe}

key: 16 15 12 19
ciphertext higher freq t

ValueError: matrix equation has no solutions